In [ ]:
%%sql
DECLARE @TodayMidnight DATETIME;
DECLARE @YesterdayMidnight DATETIME;

-- Assign today's date with time set to 00:00:00
SET @TodayMidnight = CAST(CAST(GETDATE() AS date) AS datetime);

-- Assign yesterday's midnight by subtracting one day
SET @YesterdayMidnight = DATEADD(DAY, -1, @TodayMidnight);

SELECT * 
FROM FinalMoment 
WHERE CompanyID = 130 
  AND [datetime] > @YesterdayMidnight;


In [ ]:
%%sql
SELECT CAST(CAST(GETDATE() AS date) AS datetime) AS StartOfToday;


In [ ]:
%%sql
;WITH Intervals AS
(
    SELECT 5 AS IntervalLength
UNION ALL SELECT 15
UNION ALL SELECT 60
),
CTE_Intervals AS
(
    SELECT
    fm.CompanyID,
    fm.[datetime],
    fm.Final,
    fm.[close],
    fm.volume,
    fm.MarketCap,
    I.IntervalLength,
    TimeInterval = DATEADD(MINUTE, DATEDIFF(MINUTE,0,fm.[datetime]) / I.IntervalLength * I.IntervalLength, 0),
rn_asc = ROW_NUMBER() OVER (PARTITION BY fm.CompanyID, I.IntervalLength, DATEADD(MINUTE, DATEDIFF(MINUTE,0,fm.[datetime]) / I.IntervalLength * I.IntervalLength, 0)
ORDER BY fm.[datetime] ASC),
rn_desc = ROW_NUMBER() OVER (PARTITION BY fm.CompanyID, I.IntervalLength, DATEADD(MINUTE, DATEDIFF(MINUTE,0,fm.[datetime]) / I.IntervalLength * I.IntervalLength, 0)
ORDER BY fm.[datetime] DESC)
FROM FinalMoment fm
CROSS JOIN Intervals I
WHERE fm.CompanyID = 130
AND fm.[datetime] >= DATEADD(DAY, DATEDIFF(DAY, 0, GETDATE())-1, 0) -- yesterday's midnight
AND fm.[datetime] < DATEADD(DAY, DATEDIFF(DAY, 0, GETDATE()), 0)    -- today's midnight
)
SELECT
CompanyID,
TimeInterval,
IntervalLength,
[Open] = MAX(CASE WHEN rn_asc = 1 THEN [close] END),
High = MAX([close]),
Low = MIN([close]),
[Close] = MAX(CASE WHEN rn_desc = 1 THEN [close] END),
TotalVolume = SUM(volume),
TotalMarketCap = SUM(MarketCap),
Final = MAX(CASE WHEN rn_desc = 1 THEN Final END)
FROM CTE_Intervals
GROUP BY CompanyID, TimeInterval, IntervalLength
ORDER BY CompanyID, TimeInterval, IntervalLength;


In [ ]:
%%sql
-- CREATE PROCEDURE dbo.UpsertOHLC_Summary
-- AS
BEGIN
    SET NOCOUNT ON;

    ;WITH Intervals AS
    (
        SELECT 5 AS IntervalLength
        UNION ALL SELECT 15
        UNION ALL SELECT 60
    ),
    YesterdayData AS
    (
        SELECT 
            fm.CompanyID,
            TimeInterval = DATEADD(MINUTE, (DATEDIFF(MINUTE,0,fm.[datetime]) / I.IntervalLength)*I.IntervalLength, 0),
            I.IntervalLength,
            fm.[datetime],
            fm.Final,
            fm.[close],
            fm.volume,
            fm.MarketCap,
            rn_asc = ROW_NUMBER() OVER (
                PARTITION BY fm.CompanyID, I.IntervalLength,
                             DATEADD(MINUTE, (DATEDIFF(MINUTE,0,fm.[datetime]) / I.IntervalLength)*I.IntervalLength, 0)
                ORDER BY fm.[datetime] ASC),
            rn_desc = ROW_NUMBER() OVER (
                PARTITION BY fm.CompanyID, I.IntervalLength,
                             DATEADD(MINUTE, (DATEDIFF(MINUTE,0,fm.[datetime]) / I.IntervalLength)*I.IntervalLength, 0)
                ORDER BY fm.[datetime] DESC)
        FROM FinalMoment fm
        CROSS JOIN Intervals I
        WHERE fm.[datetime] >= DATEADD(DAY, DATEDIFF(DAY, 0, GETDATE())-130, 0)  -- Yesterday's midnight
          AND fm.[datetime] < DATEADD(DAY, DATEDIFF(DAY, 0, GETDATE())-1, 0)     -- Today's midnight
    ),
    Aggregated AS
    (
        SELECT
            CompanyID,
            TimeInterval,
            IntervalLength,
            [Open] = MAX(CASE WHEN rn_asc = 1 THEN [close] END),
            High = MAX([close]),
            Low = MIN([close]),
            [Close] = MAX(CASE WHEN rn_desc = 1 THEN [close] END),
            TotalVolume = SUM(volume),
            TotalMarketCap = SUM(MarketCap),
            Final = MAX(CASE WHEN rn_desc = 1 THEN Final END)
        FROM YesterdayData
        GROUP BY CompanyID, TimeInterval, IntervalLength
    )
    MERGE OHLC_Summary AS T
    USING Aggregated AS S
        ON T.CompanyID = S.CompanyID
        AND T.TimeInterval = S.TimeInterval
        AND T.IntervalLength = S.IntervalLength
    WHEN MATCHED AND (
        T.[Open]          <> S.[Open]
        OR T.High          <> S.High
        OR T.Low           <> S.Low
        OR T.[Close]       <> S.[Close]
        OR T.TotalVolume   <> S.TotalVolume
        OR T.TotalMarketCap<> S.TotalMarketCap
        OR T.Final         <> S.Final
    )
    THEN UPDATE SET
        T.[Open]         = S.[Open],
        T.High           = S.High,
        T.Low            = S.Low,
        T.[Close]        = S.[Close],
        T.TotalVolume    = S.TotalVolume,
        T.TotalMarketCap = S.TotalMarketCap,
        T.Final          = S.Final
    WHEN NOT MATCHED BY TARGET
    THEN INSERT (
        CompanyID, TimeInterval, IntervalLength,
        [Open], High, Low, [Close],
        TotalVolume, TotalMarketCap, Final
    ) VALUES (
        S.CompanyID, S.TimeInterval, S.IntervalLength,
        S.[Open], S.High, S.Low, S.[Close],
        S.TotalVolume, S.TotalMarketCap, S.Final
    );
END

In [ ]:
%%sql
exec UpsertOHLC_Summary